# PPO Portfolio Optimization Demo

This notebook demonstrates how to use the custom **PPO-based portfolio optimizer**
modules (`DataLoader`, `PortfolioEnv`, and `PPOAgent`) for algorithmic trading.
All functionality is provided by the `traderlib.core` package without any FinRL code.


In [ ]:
from traderlib.core.data_loader import YahooDownloader

# Fetch historical data using YahooDownloader
stockData = YahooDownloader(
    ticker_list=["AAPL", "MSFT", "GOOG"],
    start_date="2022-01-03",
    end_date="2025-04-30"
).fetch_data()

stockData.head()

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

In [ ]:
import torch

import numpy as np
import pandas as pd

from torch_geometric.utils import k_hop_subgraph

from traderlib.core.data_loader import StockDataLoader
from traderlib.core.portfolio_env import PortfolioOptimizationEnv
from traderlib.core.agent import MyRLTrader
from traderlib.core.model_arch import GPM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
list_of_stocks = nasdaq_temporal["tic"].unique().tolist()
tics_in_portfolio = ["AAPL", "CMCSA", "CSCO", "FB", "HBAN", "INTC", "MSFT", "MU", "NVDA", "QQQ", "XIV"]

portfolio_nodes = []
for tic in tics_in_portfolio:
    portfolio_nodes.append(list_of_stocks.index(tic))
portfolio_nodes

In [ ]:
# reduce temporal data
nodes_kept = nodes_kept.tolist()
nasdaq_temporal["tic_id"], _ = pd.factorize(nasdaq_temporal["tic"], sort=True)
nasdaq_temporal = nasdaq_temporal[nasdaq_temporal["tic_id"].isin(nodes_kept)]
nasdaq_temporal = nasdaq_temporal.drop(columns="tic_id")
nasdaq_temporal

In [ ]:
df_portfolio = nasdaq_temporal[["day", "tic", "close", "high", "low"]]

df_portfolio_train = df_portfolio[df_portfolio["day"] < 979]
df_portfolio_test = df_portfolio[df_portfolio["day"] >= 979]

environment_train = PortfolioOptimizationEnv(
        df_portfolio_train,
        initial_amount=100000,
        comission_fee_pct=0.0025,
        time_window=50,
        features=["close", "high", "low"],
        time_column="day",
        normalize_df=None, # dataframe is already normalized
        tics_in_portfolio=tics_in_portfolio
    )

environment_test = PortfolioOptimizationEnv(
        df_portfolio_test,
        initial_amount=100000,
        comission_fee_pct=0.0025,
        time_window=50,
        features=["close", "high", "low"],
        time_column="day",
        normalize_df=None, # dataframe is already normalized
        tics_in_portfolio=tics_in_portfolio
    )

In [ ]:
MyRLTrader.run_bot_training(model, episodes=2)

In [ ]:
torch.save(model.train_policy.state_dict(), "policy_GPM.pt")

In [ ]:
UBAH_results = {
    "train": {},
    "test": {},
}

PORTFOLIO_SIZE = len(tics_in_portfolio)

# train period
terminated = False
environment_train.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_train.step(action)
UBAH_results["train"] = environment_train._asset_memory["final"]

# test period
terminated = False
environment_test.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_test.step(action)
UBAH_results["test"] = environment_test._asset_memory["final"]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(UBAH_results["train"], label="Buy and Hold")
plt.plot(GPM_results["train"], label="GPM")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in training period")
plt.legend()

plt.show()

In [ ]:
plt.plot(UBAH_results["test"], label="Buy and Hold")
plt.plot(GPM_results["test"], label="GPM")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in testing period")
plt.legend()

plt.show()